In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -qq /content/drive/MyDrive/Datasets/Apple_health.zip

In [3]:
!rm -rf train
!rm -rf test
!rm -rf val

In [4]:
import os, shutil, pathlib
import random

In [5]:
os.makedirs('train')
os.makedirs('test')
os.makedirs('val')
os.makedirs('train/Apple___Apple_scab')
os.makedirs('train/Apple___Black_rot')
os.makedirs('train/Apple___Cedar_apple_rust')
os.makedirs('train/Apple___healthy')
os.makedirs('test/Apple___Apple_scab')
os.makedirs('test/Apple___Black_rot')
os.makedirs('test/Apple___Cedar_apple_rust')
os.makedirs('test/Apple___healthy')
os.makedirs('val/Apple___Apple_scab')
os.makedirs('val/Apple___Black_rot')
os.makedirs('val/Apple___Cedar_apple_rust')
os.makedirs('val/Apple___healthy')

In [6]:
folder_paths = ["/content/test","/content/train","/content/val"]
categories = ["Apple___Apple_scab","Apple___Black_rot","Apple___Cedar_apple_rust","Apple___healthy"]

for folder_path in folder_paths:
  for category in categories:
    # Get a list of all files in the directory
    files = os.listdir(os.path.join(folder_path,category))

    # Loop through each file and delete it
    for file in files:
        file_path = os.path.join(folder_path, category, file)
        if os.path.isfile(file_path):
            os.remove(file_path)


In [7]:
random.random()

0.02034775414146972

In [8]:
for dir in ('/content/Apple___Apple_scab', '/content/Apple___Black_rot','/content/Apple___Cedar_apple_rust','/content/Apple___healthy'):
  files = os.listdir(dir)
  for f in files:
    ran_num = random.random()
    src = os.path.join(dir, f)
    # dst = '/content/train' + dir[8:] + "/" + f
    if ran_num < 0.7:
      dst = '/content/train' + dir[8:] + "/" + f
      shutil.copyfile(src=src, dst=dst)
    elif ran_num >= 0.7 and ran_num < 0.85:
      dst = '/content/val' + dir[8:] + "/" + f
      shutil.copyfile(src=src, dst=dst)
    else:
      dst = '/content/test' + dir[8:] + "/" + f
      shutil.copyfile(src=src, dst=dst)

In [9]:
from tensorflow.keras.utils import image_dataset_from_directory

new_base_dir = pathlib.Path("/content/")

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
val_dataset = image_dataset_from_directory(
    new_base_dir / "val",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 3252 files belonging to 4 classes.
Found 691 files belonging to 4 classes.
Found 702 files belonging to 4 classes.


In [10]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


In [11]:
labels_batch

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([3, 0, 3, 1, 3, 2, 2, 2, 3, 3, 0, 3, 2, 1, 3, 0, 2, 0, 1, 2, 3, 1,
       0, 3, 0, 3, 1, 2, 2, 3, 3, 3], dtype=int32)>

In [24]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling_3 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 43, 43, 64)       0         
 g2D)                                                      

In [25]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [26]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks)

Epoch 1/30
102/102 [==============================] - 8s 59ms/step - loss: 0.8264 - accuracy: 0.6756 - val_loss: 0.6106 - val_accuracy: 0.7656
Epoch 2/30
102/102 [==============================] - 5s 46ms/step - loss: 0.5185 - accuracy: 0.8087 - val_loss: 0.4029 - val_accuracy: 0.8567
Epoch 3/30
102/102 [==============================] - 6s 57ms/step - loss: 0.3332 - accuracy: 0.8795 - val_loss: 0.2353 - val_accuracy: 0.9088
Epoch 4/30
102/102 [==============================] - 5s 45ms/step - loss: 0.2334 - accuracy: 0.9148 - val_loss: 0.1626 - val_accuracy: 0.9421
Epoch 5/30
102/102 [==============================] - 6s 57ms/step - loss: 0.2350 - accuracy: 0.9148 - val_loss: 0.1992 - val_accuracy: 0.9291
Epoch 6/30
102/102 [==============================] - 5s 44ms/step - loss: 0.1720 - accuracy: 0.9354 - val_loss: 0.1706 - val_accuracy: 0.9305
Epoch 7/30
102/102 [==============================] - 5s 44ms/step - loss: 0.1503 - accuracy: 0.9462 - val_loss: 0.1562 - val_accuracy: 0.9407

In [27]:
val_loss, val_acc = model.evaluate_generator(val_dataset, steps=len(val_dataset))

# Print the validation loss and accuracy
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_acc)

<ipython-input-27-8da64b7714ee>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  val_loss, val_acc = model.evaluate_generator(val_dataset, steps=len(val_dataset))


Validation Loss: 0.02196647785604
Validation Accuracy: 0.9942113161087036


In [30]:
test_loss, test_acc = model.evaluate_generator(test_dataset, steps=len(test_dataset))

# Print the validation loss and accuracy
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

<ipython-input-30-76bc88c33a26>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_dataset, steps=len(test_dataset))


Test Loss: 0.06838461756706238
Test Accuracy: 0.9886040091514587
